In [2]:
import pandas as pd
import numpy as np
import pymc as pm
import seaborn as sns
from sklearn.model_selection import train_test_split
pd.set_option('display.max_rows', 500)
import arviz as az
import matplotlib.pyplot as plt

C:\Users\can2hr\Anaconda3\lib\site-packages\pandas\core\computation\expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
WARNING (aesara.configdefaults): g++ not available, if using conda: `conda install m2w64-toolchain`
WARNING (aesara.configdefaults): g++ not detected!  Aesara will be unable to compile C-implementations and will default to Python. Performance may be severely degraded. To remove this warning, set Aesara flags cxx to an empty string.
WARNING (aesara.tensor.blas): Using NumPy C-API based implementation for BLAS functions.
C:\Users\can2hr\Anaconda3\lib\site-packages\pkg_resources\__init__.py:123: PkgResourcesDeprecationWarning: 4.0.0-unsupported is an invalid version and will not be supported in a future release
  warnings.warn(


## READ IN DATA

In [2]:
# select all the players besides dbs and wrs
players = pd.read_csv("https://media.githubusercontent.com/media/cnickol26/BigDataBowl2023/main/nfl-big-data-bowl-2023/players.csv")

positions = ['DE', 'OLB','DT', 'ILB', 'NT', 'MLB', 'LB', 'RB', 'T', 'TE','G','QB','C','FB']
positions_df = players[players['officialPosition'].isin(positions)]
players_list = positions_df['nflId']

In [3]:
# Read in all the weeks but only for the positons above
locations = pd.DataFrame()
for i in range(1,8):
    url = 'https://media.githubusercontent.com/media/cnickol26/BigDataBowl2023/main/nfl-big-data-bowl-2023/week'+str(i)+'.csv'
    week_data = pd.read_csv(url)
    week_data = week_data[(week_data['nflId'].isin(players_list)) | (week_data['team'] == 'football')]
    locations = pd.concat([locations, week_data])

In [4]:
locations

,gameId,playId,nflId,frameId,time,jerseyNumber,team,playDirection,x,y,s,a,dis,o,dir,event
0,2021090900,97,25511.0,1,2021-09-10 00:26:31,12.0,TB,right,37.77,24.22,0.29,0.30,0.03,165.16,84.99,None
1,2021090900,97,25511.0,2,2021-09-10 00:26:31,12.0,TB,right,37.78,24.22,0.23,0.11,0.02,164.33,92.87,None
2,2021090900,97,25511.0,3,2021-09-10 00:26:31,12.0,TB,right,37.78,24.24,0.16,0.10,0.01,160.24,68.55,None
3,2021090900,97,25511.0,4,2021-09-10 00:26:31,12.0,TB,right,37.73,24.25,0.15,0.24,0.06,152.13,296.85,None
4,2021090900,97,25511.0,5,2021-09-10 00:26:31,12.0,TB,right,37.69,24.26,0.25,0.18,0.04,148.33,287.55,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
906287,2021102500,3998,NaN,71,2021-10-26 03:17:05,NaN,football,left,112.93,14.52,22.04,0.44,2.62,NaN,NaN,pass_forward
906288,2021102500,3998,NaN,72,2021-10-26 03:17:06,NaN,football,left,111.34,16.07,21.95,1.40,2.22,NaN,NaN,None
906289,2021102500,3998,NaN,73,2021-10-26 03:17:06,NaN,football,left,109.74,17.60,21.81,2.02,2.21,NaN,NaN,None
906290,2021102500,3998,NaN,74,2021-10-26 03:17:06,NaN,football,left,108.17,19.12,21.66,2.39,2.19,NaN,NaN,None


In [5]:
pff = pd.read_csv("https://media.githubusercontent.com/media/cnickol26/BigDataBowl2023/main/nfl-big-data-bowl-2023/pffScoutingData.csv")

In [6]:
pff

,gameId,playId,nflId,pff_role,pff_positionLinedUp,pff_hit,pff_hurry,pff_sack,pff_beatenByDefender,pff_hitAllowed,pff_hurryAllowed,pff_sackAllowed,pff_nflIdBlockedPlayer,pff_blockType,pff_backFieldBlock
0,2021090900,97,25511,Pass,QB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021090900,97,35481,Pass Route,TE-L,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021090900,97,35634,Pass Route,LWR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2021090900,97,39985,Pass Route,HB-R,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2021090900,97,40151,Pass Block,C,NaN,NaN,NaN,0.0,0.0,0.0,0.0,44955.0,SW,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188249,2021110100,4433,52507,Pass Block,LT,NaN,NaN,NaN,0.0,0.0,1.0,0.0,43338.0,PP,0.0
188250,2021110100,4433,52546,Coverage,SCBoR,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
188251,2021110100,4433,52573,Pass Route,SLoWR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
188252,2021110100,4433,52585,Pass Rush,LEO,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
# Going to join these columns in with the tracking data
pff_filter = pff[['gameId','playId','nflId','pff_role','pff_positionLinedUp','pff_nflIdBlockedPlayer','pff_blockType']]

In [8]:
locations_ball = locations.merge(pff_filter, on=['gameId','playId','nflId'], how='left')

In [9]:
plays = pd.read_csv("https://media.githubusercontent.com/media/cnickol26/BigDataBowl2023/main/nfl-big-data-bowl-2023/plays.csv")

In [10]:
plays

,gameId,playId,playDescription,quarter,down,yardsToGo,possessionTeam,defensiveTeam,yardlineSide,yardlineNumber,...,foulNFLId3,absoluteYardlineNumber,offenseFormation,personnelO,defendersInBox,personnelD,dropBackType,pff_playAction,pff_passCoverage,pff_passCoverageType
0,2021090900,97,(13:33) (Shotgun) T.Brady pass incomplete deep...,1,3,2,TB,DAL,TB,33,...,NaN,43.0,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,"4 DL, 2 LB, 5 DB",TRADITIONAL,0,Cover-1,Man
1,2021090900,137,(13:18) (Shotgun) D.Prescott pass deep left to...,1,1,10,DAL,TB,DAL,2,...,NaN,108.0,EMPTY,"1 RB, 2 TE, 2 WR",6.0,"4 DL, 4 LB, 3 DB",TRADITIONAL,0,Cover-3,Zone
2,2021090900,187,(12:23) (Shotgun) D.Prescott pass short middle...,1,2,6,DAL,TB,DAL,34,...,NaN,76.0,SHOTGUN,"0 RB, 2 TE, 3 WR",6.0,"3 DL, 3 LB, 5 DB",TRADITIONAL,0,Cover-3,Zone
3,2021090900,282,(9:56) D.Prescott pass incomplete deep left to...,1,1,10,DAL,TB,TB,39,...,NaN,49.0,SINGLEBACK,"1 RB, 2 TE, 2 WR",6.0,"4 DL, 3 LB, 4 DB",TRADITIONAL,1,Cover-3,Zone
4,2021090900,349,(9:46) (Shotgun) D.Prescott pass incomplete sh...,1,3,15,DAL,TB,TB,44,...,NaN,54.0,SHOTGUN,"1 RB, 1 TE, 3 WR",7.0,"3 DL, 4 LB, 4 DB",TRADITIONAL,0,Cover-3,Zone
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8553,2021110100,4310,(1:56) (Shotgun) P.Mahomes sacked at NYG 16 fo...,4,3,8,KC,NYG,NYG,8,...,NaN,18.0,SHOTGUN,"1 RB, 1 TE, 3 WR",4.0,"1 DL, 3 LB, 7 DB",SCRAMBLE,0,Bracket,Other
8554,2021110100,4363,(1:07) (Shotgun) D.Jones pass short right to E...,4,1,10,NYG,KC,NYG,25,...,NaN,35.0,SHOTGUN,"1 RB, 1 TE, 3 WR",5.0,"4 DL, 1 LB, 6 DB",SCRAMBLE,0,Cover-2,Zone
8555,2021110100,4392,"(1:01) (No Huddle, Shotgun) D.Jones sacked at ...",4,2,7,NYG,KC,NYG,28,...,NaN,38.0,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,"4 DL, 1 LB, 6 DB",TRADITIONAL,0,Cover-2,Zone
8556,2021110100,4411,"(:39) (No Huddle, Shotgun) D.Jones pass incomp...",4,3,15,NYG,KC,NYG,20,...,NaN,30.0,SHOTGUN,"1 RB, 1 TE, 3 WR",5.0,"4 DL, 1 LB, 6 DB",TRADITIONAL,0,Cover-2,Zone


In [11]:
# Also going to be added in to the tracking data
plays_filter = plays[['gameId','playId','dropBackType','pff_playAction']]

In [12]:
locations_ball = locations_ball.merge(plays_filter, on=['gameId','playId'], how='left')

In [13]:
locations_ball

,gameId,playId,nflId,frameId,time,jerseyNumber,team,playDirection,x,y,...,dis,o,dir,event,pff_role,pff_positionLinedUp,pff_nflIdBlockedPlayer,pff_blockType,dropBackType,pff_playAction
0,2021090900,97,25511.0,1,2021-09-10 00:26:31,12.0,TB,right,37.77,24.22,...,0.03,165.16,84.99,None,Pass,QB,NaN,NaN,TRADITIONAL,0
1,2021090900,97,25511.0,2,2021-09-10 00:26:31,12.0,TB,right,37.78,24.22,...,0.02,164.33,92.87,None,Pass,QB,NaN,NaN,TRADITIONAL,0
2,2021090900,97,25511.0,3,2021-09-10 00:26:31,12.0,TB,right,37.78,24.24,...,0.01,160.24,68.55,None,Pass,QB,NaN,NaN,TRADITIONAL,0
3,2021090900,97,25511.0,4,2021-09-10 00:26:31,12.0,TB,right,37.73,24.25,...,0.06,152.13,296.85,None,Pass,QB,NaN,NaN,TRADITIONAL,0
4,2021090900,97,25511.0,5,2021-09-10 00:26:31,12.0,TB,right,37.69,24.26,...,0.04,148.33,287.55,None,Pass,QB,NaN,NaN,TRADITIONAL,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4863391,2021102500,3998,NaN,71,2021-10-26 03:17:05,NaN,football,left,112.93,14.52,...,2.62,NaN,NaN,pass_forward,NaN,NaN,NaN,NaN,SCRAMBLE,0
4863392,2021102500,3998,NaN,72,2021-10-26 03:17:06,NaN,football,left,111.34,16.07,...,2.22,NaN,NaN,None,NaN,NaN,NaN,NaN,SCRAMBLE,0
4863393,2021102500,3998,NaN,73,2021-10-26 03:17:06,NaN,football,left,109.74,17.60,...,2.21,NaN,NaN,None,NaN,NaN,NaN,NaN,SCRAMBLE,0
4863394,2021102500,3998,NaN,74,2021-10-26 03:17:06,NaN,football,left,108.17,19.12,...,2.19,NaN,NaN,None,NaN,NaN,NaN,NaN,SCRAMBLE,0


In [14]:
# create unique play ID for each play
locations_ball['uniqueplayId'] = locations_ball['gameId'].astype(str) + locations_ball['playId'].astype(str)

In [15]:
# Dropping all of the rows that occur before the snap of the ball
snap_time = locations_ball.loc[locations_ball['event'] == 'ball_snap',['uniqueplayId', 'time']].rename(columns = {'time':'ball_snap_time'})
snap_time_unique = snap_time.drop_duplicates(subset = 'uniqueplayId')

In [16]:
locations_ball2 = locations_ball.merge(snap_time_unique, on = 'uniqueplayId', how = 'left')
locations_ball2

,gameId,playId,nflId,frameId,time,jerseyNumber,team,playDirection,x,y,...,dir,event,pff_role,pff_positionLinedUp,pff_nflIdBlockedPlayer,pff_blockType,dropBackType,pff_playAction,uniqueplayId,ball_snap_time
0,2021090900,97,25511.0,1,2021-09-10 00:26:31,12.0,TB,right,37.77,24.22,...,84.99,None,Pass,QB,NaN,NaN,TRADITIONAL,0,202109090097,2021-09-10 00:26:31
1,2021090900,97,25511.0,2,2021-09-10 00:26:31,12.0,TB,right,37.78,24.22,...,92.87,None,Pass,QB,NaN,NaN,TRADITIONAL,0,202109090097,2021-09-10 00:26:31
2,2021090900,97,25511.0,3,2021-09-10 00:26:31,12.0,TB,right,37.78,24.24,...,68.55,None,Pass,QB,NaN,NaN,TRADITIONAL,0,202109090097,2021-09-10 00:26:31
3,2021090900,97,25511.0,4,2021-09-10 00:26:31,12.0,TB,right,37.73,24.25,...,296.85,None,Pass,QB,NaN,NaN,TRADITIONAL,0,202109090097,2021-09-10 00:26:31
4,2021090900,97,25511.0,5,2021-09-10 00:26:31,12.0,TB,right,37.69,24.26,...,287.55,None,Pass,QB,NaN,NaN,TRADITIONAL,0,202109090097,2021-09-10 00:26:31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4863391,2021102500,3998,NaN,71,2021-10-26 03:17:05,NaN,football,left,112.93,14.52,...,NaN,pass_forward,NaN,NaN,NaN,NaN,SCRAMBLE,0,20211025003998,2021-10-26 03:16:59
4863392,2021102500,3998,NaN,72,2021-10-26 03:17:06,NaN,football,left,111.34,16.07,...,NaN,None,NaN,NaN,NaN,NaN,SCRAMBLE,0,20211025003998,2021-10-26 03:16:59
4863393,2021102500,3998,NaN,73,2021-10-26 03:17:06,NaN,football,left,109.74,17.60,...,NaN,None,NaN,NaN,NaN,NaN,SCRAMBLE,0,20211025003998,2021-10-26 03:16:59
4863394,2021102500,3998,NaN,74,2021-10-26 03:17:06,NaN,football,left,108.17,19.12,...,NaN,None,NaN,NaN,NaN,NaN,SCRAMBLE,0,20211025003998,2021-10-26 03:16:59


In [18]:
locations_ball3 = locations_ball2[~(locations_ball2['time'] < locations_ball2['ball_snap_time'])]

In [19]:
locations_ball3

,gameId,playId,nflId,frameId,time,jerseyNumber,team,playDirection,x,y,...,dir,event,pff_role,pff_positionLinedUp,pff_nflIdBlockedPlayer,pff_blockType,dropBackType,pff_playAction,uniqueplayId,ball_snap_time
0,2021090900,97,25511.0,1,2021-09-10 00:26:31,12.0,TB,right,37.77,24.22,...,84.99,None,Pass,QB,NaN,NaN,TRADITIONAL,0,202109090097,2021-09-10 00:26:31
1,2021090900,97,25511.0,2,2021-09-10 00:26:31,12.0,TB,right,37.78,24.22,...,92.87,None,Pass,QB,NaN,NaN,TRADITIONAL,0,202109090097,2021-09-10 00:26:31
2,2021090900,97,25511.0,3,2021-09-10 00:26:31,12.0,TB,right,37.78,24.24,...,68.55,None,Pass,QB,NaN,NaN,TRADITIONAL,0,202109090097,2021-09-10 00:26:31
3,2021090900,97,25511.0,4,2021-09-10 00:26:31,12.0,TB,right,37.73,24.25,...,296.85,None,Pass,QB,NaN,NaN,TRADITIONAL,0,202109090097,2021-09-10 00:26:31
4,2021090900,97,25511.0,5,2021-09-10 00:26:31,12.0,TB,right,37.69,24.26,...,287.55,None,Pass,QB,NaN,NaN,TRADITIONAL,0,202109090097,2021-09-10 00:26:31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4863391,2021102500,3998,NaN,71,2021-10-26 03:17:05,NaN,football,left,112.93,14.52,...,NaN,pass_forward,NaN,NaN,NaN,NaN,SCRAMBLE,0,20211025003998,2021-10-26 03:16:59
4863392,2021102500,3998,NaN,72,2021-10-26 03:17:06,NaN,football,left,111.34,16.07,...,NaN,None,NaN,NaN,NaN,NaN,SCRAMBLE,0,20211025003998,2021-10-26 03:16:59
4863393,2021102500,3998,NaN,73,2021-10-26 03:17:06,NaN,football,left,109.74,17.60,...,NaN,None,NaN,NaN,NaN,NaN,SCRAMBLE,0,20211025003998,2021-10-26 03:16:59
4863394,2021102500,3998,NaN,74,2021-10-26 03:17:06,NaN,football,left,108.17,19.12,...,NaN,None,NaN,NaN,NaN,NaN,SCRAMBLE,0,20211025003998,2021-10-26 03:16:59


In [20]:
# Add the location of the football on the play to each row then standardize x and y on this location
locations_ball4 = locations_ball3.loc[(locations_ball3['team'] == 'football') & (locations_ball3['event'] == 'ball_snap'),
                               ['uniqueplayId', 'x', 'y']
                               ].rename(columns={'x':'football_x',
                                                 'y':'football_y'}).merge(locations_ball3, on='uniqueplayId', how='left')
locations_ball4['new_x'] = abs(locations_ball4['x']-locations_ball4['football_x'])
locations_ball4['new_y'] = np.where(locations_ball4['playDirection']=='right', 
                                   locations_ball4['football_y']-locations_ball4['y'],
                                   locations_ball4['y']-locations_ball4['football_y'])

In [21]:
locations_ball4

,uniqueplayId,football_x,football_y,gameId,playId,nflId,frameId,time,jerseyNumber,team,...,event,pff_role,pff_positionLinedUp,pff_nflIdBlockedPlayer,pff_blockType,dropBackType,pff_playAction,ball_snap_time,new_x,new_y
0,202109090097,41.56,23.92,2021090900,97,25511.0,1,2021-09-10 00:26:31,12.0,TB,...,None,Pass,QB,NaN,NaN,TRADITIONAL,0,2021-09-10 00:26:31,3.79,-0.30
1,202109090097,41.56,23.92,2021090900,97,25511.0,2,2021-09-10 00:26:31,12.0,TB,...,None,Pass,QB,NaN,NaN,TRADITIONAL,0,2021-09-10 00:26:31,3.78,-0.30
2,202109090097,41.56,23.92,2021090900,97,25511.0,3,2021-09-10 00:26:31,12.0,TB,...,None,Pass,QB,NaN,NaN,TRADITIONAL,0,2021-09-10 00:26:31,3.78,-0.32
3,202109090097,41.56,23.92,2021090900,97,25511.0,4,2021-09-10 00:26:31,12.0,TB,...,None,Pass,QB,NaN,NaN,TRADITIONAL,0,2021-09-10 00:26:31,3.83,-0.33
4,202109090097,41.56,23.92,2021090900,97,25511.0,5,2021-09-10 00:26:31,12.0,TB,...,None,Pass,QB,NaN,NaN,TRADITIONAL,0,2021-09-10 00:26:31,3.87,-0.34
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4625587,20211025003998,105.09,23.78,2021102500,3998,NaN,71,2021-10-26 03:17:05,NaN,football,...,pass_forward,NaN,NaN,NaN,NaN,SCRAMBLE,0,2021-10-26 03:16:59,7.84,-9.26
4625588,20211025003998,105.09,23.78,2021102500,3998,NaN,72,2021-10-26 03:17:06,NaN,football,...,None,NaN,NaN,NaN,NaN,SCRAMBLE,0,2021-10-26 03:16:59,6.25,-7.71
4625589,20211025003998,105.09,23.78,2021102500,3998,NaN,73,2021-10-26 03:17:06,NaN,football,...,None,NaN,NaN,NaN,NaN,SCRAMBLE,0,2021-10-26 03:16:59,4.65,-6.18
4625590,20211025003998,105.09,23.78,2021102500,3998,NaN,74,2021-10-26 03:17:06,NaN,football,...,None,NaN,NaN,NaN,NaN,SCRAMBLE,0,2021-10-26 03:16:59,3.08,-4.66


In [22]:
## Filtering to only be traditional dropbacks
locations_ball5 = locations_ball4[locations_ball4['dropBackType'] == 'TRADITIONAL']

### REMOVING CHIP BLOCK

In [23]:
locations_ball5['chip_side'] = np.where(locations_ball5['pff_blockType']=='CH', np.where(locations_ball5['new_y']>0, 'right', 'left'), 'none')
locations_ball5['chip_side'] = locations_ball5.groupby('uniqueplayId')['chip_side'].transform(lambda x: 'left' if 'left' in x.unique() else 'right' if 'right' in x.unique() else 'none')

<ipython-input-23-c7c9d805934c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  locations_ball5['chip_side'] = np.where(locations_ball5['pff_blockType']=='CH', np.where(locations_ball5['new_y']>0, 'right', 'left'), 'none')
<ipython-input-23-c7c9d805934c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  locations_ball5['chip_side'] = locations_ball5.groupby('uniqueplayId')['chip_side'].transform(lambda x: 'left' if 'left' in x.unique() else 'right' if 'right' in x.unique() else 'none')


In [24]:
# Remove the offensive linesmen on the side of the field where the chip is 
locations_no_chip = locations_ball5.loc[~(((locations_ball5['pff_positionLinedUp']=='LT')& (locations_ball5['chip_side']=='left'))|
                                ((locations_ball5['pff_positionLinedUp']=='RT')& (locations_ball5['chip_side']=='right')))]

In [25]:
# Take out the rbs, tes, fbs
chip_positions = ['HB-L','HB-R','TE-R','HB','TE-L','TE-oR','TE-iR','TE-iL','TE-oL','FB-R','FB-L','FB']
locations_no_chip = locations_no_chip[~(locations_no_chip['pff_positionLinedUp'].isin(chip_positions))]

In [26]:
## Now going to filter out anything after the ball has been thrown
end_time = locations_no_chip.loc[locations_no_chip['event'].isin(['pass_forward','qb_sack','qb_strip_sack']),['uniqueplayId', 'time']].rename(columns = {'time':'end_time'})
end_time_unique = end_time.drop_duplicates(subset = 'uniqueplayId')

In [27]:
locations_final = locations_no_chip.merge(end_time_unique, on = 'uniqueplayId', how = 'left')
locations_final = locations_final.dropna(subset = ['end_time'])

In [28]:
# Left with only plays that resulted in a sack, strip sack, or throw by the qb
locations_final

,uniqueplayId,football_x,football_y,gameId,playId,nflId,frameId,time,jerseyNumber,team,...,pff_positionLinedUp,pff_nflIdBlockedPlayer,pff_blockType,dropBackType,pff_playAction,ball_snap_time,new_x,new_y,chip_side,end_time
0,202109090097,41.56,23.92,2021090900,97,25511.0,1,2021-09-10 00:26:31,12.0,TB,...,QB,NaN,NaN,TRADITIONAL,0,2021-09-10 00:26:31,3.79,-0.30,none,2021-09-10 00:26:35
1,202109090097,41.56,23.92,2021090900,97,25511.0,2,2021-09-10 00:26:31,12.0,TB,...,QB,NaN,NaN,TRADITIONAL,0,2021-09-10 00:26:31,3.78,-0.30,none,2021-09-10 00:26:35
2,202109090097,41.56,23.92,2021090900,97,25511.0,3,2021-09-10 00:26:31,12.0,TB,...,QB,NaN,NaN,TRADITIONAL,0,2021-09-10 00:26:31,3.78,-0.32,none,2021-09-10 00:26:35
3,202109090097,41.56,23.92,2021090900,97,25511.0,4,2021-09-10 00:26:31,12.0,TB,...,QB,NaN,NaN,TRADITIONAL,0,2021-09-10 00:26:31,3.83,-0.33,none,2021-09-10 00:26:35
4,202109090097,41.56,23.92,2021090900,97,25511.0,5,2021-09-10 00:26:31,12.0,TB,...,QB,NaN,NaN,TRADITIONAL,0,2021-09-10 00:26:31,3.87,-0.34,none,2021-09-10 00:26:35
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2897733,20211025003926,85.95,23.68,2021102500,3926,NaN,45,2021-10-26 03:14:08,NaN,football,...,NaN,NaN,NaN,TRADITIONAL,0,2021-10-26 03:14:05,7.18,3.67,none,2021-10-26 03:14:08
2897734,20211025003926,85.95,23.68,2021102500,3926,NaN,46,2021-10-26 03:14:08,NaN,football,...,NaN,NaN,NaN,TRADITIONAL,0,2021-10-26 03:14:05,7.22,3.89,none,2021-10-26 03:14:08
2897735,20211025003926,85.95,23.68,2021102500,3926,NaN,47,2021-10-26 03:14:09,NaN,football,...,NaN,NaN,NaN,TRADITIONAL,0,2021-10-26 03:14:05,7.27,4.08,none,2021-10-26 03:14:08
2897736,20211025003926,85.95,23.68,2021102500,3926,NaN,48,2021-10-26 03:14:09,NaN,football,...,NaN,NaN,NaN,TRADITIONAL,0,2021-10-26 03:14:05,7.31,4.26,none,2021-10-26 03:14:08


In [34]:
# Now going to remove any players that aren't the tackles, defensive ends, qb or ball
subset1 = locations_final[['pff_nflIdBlockedPlayer', 'uniqueplayId','pff_positionLinedUp']]
subset2 = subset1[subset1['pff_positionLinedUp'].isin(['LT','RT'])]

In [39]:
# left here with a list of each who each tackle blocked on each play
subset3 = subset2.drop_duplicates(['uniqueplayId', 'pff_positionLinedUp'])
subset3

,pff_nflIdBlockedPlayer,uniqueplayId,pff_positionLinedUp
129,42403.0,202109090097,LT
387,53441.0,202109090097,RT
631,35454.0,2021090900137,LT
775,40074.0,2021090900137,RT
1269,35454.0,2021090900187,LT
...,...,...,...
2896131,41786.0,20211025003684,RT
2896661,46083.0,20211025003904,LT
2896883,37097.0,20211025003904,RT
2897179,46083.0,20211025003926,LT


In [44]:
left_ends = subset3[subset3['pff_positionLinedUp'] == 'LT'].rename(columns = {'pff_nflIdBlockedPlayer':'Left_End'}).drop(['pff_positionLinedUp'],axis = 1)
left_ends

,Left_End,uniqueplayId
129,42403.0,202109090097
631,35454.0,2021090900137
1269,35454.0,2021090900187
1750,34777.0,2021090900282
2292,46081.0,2021090900349
...,...,...
2894764,52556.0,20211025003536
2895490,52556.0,20211025003660
2895972,52556.0,20211025003684
2896661,46083.0,20211025003904


In [45]:
right_ends = subset3[subset3['pff_positionLinedUp'] == 'RT'].rename(columns = {'pff_nflIdBlockedPlayer':'Right_End'}).drop(['pff_positionLinedUp'],axis = 1)
right_ends

,Right_End,uniqueplayId
387,53441.0,202109090097
775,40074.0,2021090900137
1362,35441.0,2021090900187
1945,40074.0,2021090900282
2949,41263.0,2021090900410
...,...,...
2894953,35562.0,20211025003536
2895589,41786.0,20211025003660
2896131,41786.0,20211025003684
2896883,37097.0,20211025003904


In [47]:
locations_right = locations_final.merge(right_ends, on = 'uniqueplayId', how = 'left')
locations_right

,uniqueplayId,football_x,football_y,gameId,playId,nflId,frameId,time,jerseyNumber,team,...,pff_nflIdBlockedPlayer,pff_blockType,dropBackType,pff_playAction,ball_snap_time,new_x,new_y,chip_side,end_time,Right_End
0,202109090097,41.56,23.92,2021090900,97,25511.0,1,2021-09-10 00:26:31,12.0,TB,...,NaN,NaN,TRADITIONAL,0,2021-09-10 00:26:31,3.79,-0.30,none,2021-09-10 00:26:35,53441.0
1,202109090097,41.56,23.92,2021090900,97,25511.0,2,2021-09-10 00:26:31,12.0,TB,...,NaN,NaN,TRADITIONAL,0,2021-09-10 00:26:31,3.78,-0.30,none,2021-09-10 00:26:35,53441.0
2,202109090097,41.56,23.92,2021090900,97,25511.0,3,2021-09-10 00:26:31,12.0,TB,...,NaN,NaN,TRADITIONAL,0,2021-09-10 00:26:31,3.78,-0.32,none,2021-09-10 00:26:35,53441.0
3,202109090097,41.56,23.92,2021090900,97,25511.0,4,2021-09-10 00:26:31,12.0,TB,...,NaN,NaN,TRADITIONAL,0,2021-09-10 00:26:31,3.83,-0.33,none,2021-09-10 00:26:35,53441.0
4,202109090097,41.56,23.92,2021090900,97,25511.0,5,2021-09-10 00:26:31,12.0,TB,...,NaN,NaN,TRADITIONAL,0,2021-09-10 00:26:31,3.87,-0.34,none,2021-09-10 00:26:35,53441.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2884971,20211025003926,85.95,23.68,2021102500,3926,NaN,45,2021-10-26 03:14:08,NaN,football,...,NaN,NaN,TRADITIONAL,0,2021-10-26 03:14:05,7.18,3.67,none,2021-10-26 03:14:08,37097.0
2884972,20211025003926,85.95,23.68,2021102500,3926,NaN,46,2021-10-26 03:14:08,NaN,football,...,NaN,NaN,TRADITIONAL,0,2021-10-26 03:14:05,7.22,3.89,none,2021-10-26 03:14:08,37097.0
2884973,20211025003926,85.95,23.68,2021102500,3926,NaN,47,2021-10-26 03:14:09,NaN,football,...,NaN,NaN,TRADITIONAL,0,2021-10-26 03:14:05,7.27,4.08,none,2021-10-26 03:14:08,37097.0
2884974,20211025003926,85.95,23.68,2021102500,3926,NaN,48,2021-10-26 03:14:09,NaN,football,...,NaN,NaN,TRADITIONAL,0,2021-10-26 03:14:05,7.31,4.26,none,2021-10-26 03:14:08,37097.0


In [79]:
locations_left = locations_right.merge(left_ends, on = 'uniqueplayId', how = 'left')
locations_left['pff_positionLinedUp'] = locations_left['pff_positionLinedUp'].fillna('ball')

In [81]:
locations_non_end = locations_left[locations_left['pff_positionLinedUp'].isin(['LT','RT','QB','ball'])]
locations_end = locations_left[((locations_left['nflId'] == locations_left['Left_End'])|(locations_left['nflId'] == locations_left['Right_End']))]

In [82]:
locations_final2 = locations_end.merge(locations_non_end, how = 'outer')

<ipython-input-82-672d86c8203d>:1: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  locations_final2 = locations_end.merge(locations_non_end, how = 'outer')


In [83]:
locations_final2

,uniqueplayId,football_x,football_y,gameId,playId,nflId,frameId,time,jerseyNumber,team,...,pff_blockType,dropBackType,pff_playAction,ball_snap_time,new_x,new_y,chip_side,end_time,Right_End,Left_End
0,202109090097,41.56,23.92,2021090900,97,42403.0,1,2021-09-10 00:26:31,94.0,DAL,...,NaN,TRADITIONAL,0,2021-09-10 00:26:31,2.47,-8.77,none,2021-09-10 00:26:35,53441.0,42403.0
1,202109090097,41.56,23.92,2021090900,97,42403.0,2,2021-09-10 00:26:31,94.0,DAL,...,NaN,TRADITIONAL,0,2021-09-10 00:26:31,2.46,-8.76,none,2021-09-10 00:26:35,53441.0,42403.0
2,202109090097,41.56,23.92,2021090900,97,42403.0,3,2021-09-10 00:26:31,94.0,DAL,...,NaN,TRADITIONAL,0,2021-09-10 00:26:31,2.43,-8.75,none,2021-09-10 00:26:35,53441.0,42403.0
3,202109090097,41.56,23.92,2021090900,97,42403.0,4,2021-09-10 00:26:31,94.0,DAL,...,NaN,TRADITIONAL,0,2021-09-10 00:26:31,2.42,-8.74,none,2021-09-10 00:26:35,53441.0,42403.0
4,202109090097,41.56,23.92,2021090900,97,42403.0,5,2021-09-10 00:26:31,94.0,DAL,...,NaN,TRADITIONAL,0,2021-09-10 00:26:31,2.40,-8.73,none,2021-09-10 00:26:35,53441.0,42403.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1203714,20211025003926,85.95,23.68,2021102500,3926,NaN,45,2021-10-26 03:14:08,NaN,football,...,NaN,TRADITIONAL,0,2021-10-26 03:14:05,7.18,3.67,none,2021-10-26 03:14:08,37097.0,46083.0
1203715,20211025003926,85.95,23.68,2021102500,3926,NaN,46,2021-10-26 03:14:08,NaN,football,...,NaN,TRADITIONAL,0,2021-10-26 03:14:05,7.22,3.89,none,2021-10-26 03:14:08,37097.0,46083.0
1203716,20211025003926,85.95,23.68,2021102500,3926,NaN,47,2021-10-26 03:14:09,NaN,football,...,NaN,TRADITIONAL,0,2021-10-26 03:14:05,7.27,4.08,none,2021-10-26 03:14:08,37097.0,46083.0
1203717,20211025003926,85.95,23.68,2021102500,3926,NaN,48,2021-10-26 03:14:09,NaN,football,...,NaN,TRADITIONAL,0,2021-10-26 03:14:05,7.31,4.26,none,2021-10-26 03:14:08,37097.0,46083.0


In [119]:
#locations_final2.to_csv('data_not_flipped.csv', index = False)

In [84]:
### Next going to want to flip one of the ends to standardize

In [85]:
locations_final2['pff_positionLinedUp'].value_counts()

QB       212784
ball     212776
RT       200781
LT       192996
LEO       85097
REO       79903
ROLB      67896
LOLB      62347
LE        26442
RE        20448
DLT       17574
DRT       17049
LILB       1628
NT         1045
LLB        1014
RILB        869
NRT         711
RLB         702
NLT         675
SCBR        347
SCBL        227
MLB         221
SCBiL        81
SCBiR        42
RCB          32
SCBoL        32
Name: pff_positionLinedUp, dtype: int64

In [90]:
loc_subset = locations_final2[((locations_final2['event'] == 'ball_snap') & (locations_final2['new_y'] > 0))]

In [99]:
loc_subset2 = loc_subset[['uniqueplayId','nflId']]
loc_subset2['Flip'] = 1

<ipython-input-99-245193814b07>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  loc_subset2['Flip'] = 1


In [108]:
locations_flip = locations_final2.merge(loc_subset2, on = ['uniqueplayId','nflId'], how = 'left')
locations_flip['Flip'] = locations_flip['Flip'].fillna(0)
locations_flip['Flip'].value_counts()

0.0    729372
1.0    474347
Name: Flip, dtype: int64

In [110]:
locations_flip2 = locations_flip[((locations_flip['Flip'] == 1) | locations_flip['pff_positionLinedUp'].isin(['QB','ball']))]
locations_flip2

,uniqueplayId,football_x,football_y,gameId,playId,nflId,frameId,time,jerseyNumber,team,...,dropBackType,pff_playAction,ball_snap_time,new_x,new_y,chip_side,end_time,Right_End,Left_End,Flip
43,202109090097,41.56,23.92,2021090900,97,53441.0,1,2021-09-10 00:26:31,11.0,DAL,...,TRADITIONAL,0,2021-09-10 00:26:31,2.15,2.04,none,2021-09-10 00:26:35,53441.0,42403.0,1.0
44,202109090097,41.56,23.92,2021090900,97,53441.0,2,2021-09-10 00:26:31,11.0,DAL,...,TRADITIONAL,0,2021-09-10 00:26:31,2.15,2.04,none,2021-09-10 00:26:35,53441.0,42403.0,1.0
45,202109090097,41.56,23.92,2021090900,97,53441.0,3,2021-09-10 00:26:31,11.0,DAL,...,TRADITIONAL,0,2021-09-10 00:26:31,2.15,2.03,none,2021-09-10 00:26:35,53441.0,42403.0,1.0
46,202109090097,41.56,23.92,2021090900,97,53441.0,4,2021-09-10 00:26:31,11.0,DAL,...,TRADITIONAL,0,2021-09-10 00:26:31,2.14,2.01,none,2021-09-10 00:26:35,53441.0,42403.0,1.0
47,202109090097,41.56,23.92,2021090900,97,53441.0,5,2021-09-10 00:26:31,11.0,DAL,...,TRADITIONAL,0,2021-09-10 00:26:31,2.13,2.00,none,2021-09-10 00:26:35,53441.0,42403.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1203714,20211025003926,85.95,23.68,2021102500,3926,NaN,45,2021-10-26 03:14:08,NaN,football,...,TRADITIONAL,0,2021-10-26 03:14:05,7.18,3.67,none,2021-10-26 03:14:08,37097.0,46083.0,0.0
1203715,20211025003926,85.95,23.68,2021102500,3926,NaN,46,2021-10-26 03:14:08,NaN,football,...,TRADITIONAL,0,2021-10-26 03:14:05,7.22,3.89,none,2021-10-26 03:14:08,37097.0,46083.0,0.0
1203716,20211025003926,85.95,23.68,2021102500,3926,NaN,47,2021-10-26 03:14:09,NaN,football,...,TRADITIONAL,0,2021-10-26 03:14:05,7.27,4.08,none,2021-10-26 03:14:08,37097.0,46083.0,0.0
1203717,20211025003926,85.95,23.68,2021102500,3926,NaN,48,2021-10-26 03:14:09,NaN,football,...,TRADITIONAL,0,2021-10-26 03:14:05,7.31,4.26,none,2021-10-26 03:14:08,37097.0,46083.0,0.0


In [111]:
locations_flip2['pff_positionLinedUp'] = locations_flip2['pff_positionLinedUp'].replace({'QB': 'QB_flip', 'ball': 'ball_flip'})

<ipython-input-111-0108b031e214>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  locations_flip2['pff_positionLinedUp'] = locations_flip2['pff_positionLinedUp'].replace({'QB': 'QB_flip', 'ball': 'ball_flip'})


In [112]:
locations_flip2['pff_positionLinedUp'].value_counts()

QB_flip      212784
ball_flip    212776
RT           200710
LEO           85060
LOLB          62347
LE            26442
DLT           17574
LILB           1628
LLB            1014
NLT             641
NT              293
SCBL            227
MLB             112
SCBiL            81
SCBoL            32
Name: pff_positionLinedUp, dtype: int64

In [113]:
locations_flip['new_y'] = locations_flip['new_y'] * -1

In [114]:
locations_flip['new_y'].value_counts()

-0.00     24818
 0.02      5467
-0.02      5359
 0.09      4815
 0.07      4519
          ...  
 11.04        1
 10.75        1
 10.45        1
 10.68        1
 9.25         1
Name: new_y, Length: 7223, dtype: int64

In [115]:
locations_same = locations_flip[((locations_flip['Flip'] == 0) | locations_flip['pff_positionLinedUp'].isin(['QB','ball']))]

In [116]:
locations_with_flip = pd.concat([locations_same, locations_flip2])

In [117]:
locations_with_flip

,uniqueplayId,football_x,football_y,gameId,playId,nflId,frameId,time,jerseyNumber,team,...,dropBackType,pff_playAction,ball_snap_time,new_x,new_y,chip_side,end_time,Right_End,Left_End,Flip
0,202109090097,41.56,23.92,2021090900,97,42403.0,1,2021-09-10 00:26:31,94.0,DAL,...,TRADITIONAL,0,2021-09-10 00:26:31,2.47,8.77,none,2021-09-10 00:26:35,53441.0,42403.0,0.0
1,202109090097,41.56,23.92,2021090900,97,42403.0,2,2021-09-10 00:26:31,94.0,DAL,...,TRADITIONAL,0,2021-09-10 00:26:31,2.46,8.76,none,2021-09-10 00:26:35,53441.0,42403.0,0.0
2,202109090097,41.56,23.92,2021090900,97,42403.0,3,2021-09-10 00:26:31,94.0,DAL,...,TRADITIONAL,0,2021-09-10 00:26:31,2.43,8.75,none,2021-09-10 00:26:35,53441.0,42403.0,0.0
3,202109090097,41.56,23.92,2021090900,97,42403.0,4,2021-09-10 00:26:31,94.0,DAL,...,TRADITIONAL,0,2021-09-10 00:26:31,2.42,8.74,none,2021-09-10 00:26:35,53441.0,42403.0,0.0
4,202109090097,41.56,23.92,2021090900,97,42403.0,5,2021-09-10 00:26:31,94.0,DAL,...,TRADITIONAL,0,2021-09-10 00:26:31,2.40,8.73,none,2021-09-10 00:26:35,53441.0,42403.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1203714,20211025003926,85.95,23.68,2021102500,3926,NaN,45,2021-10-26 03:14:08,NaN,football,...,TRADITIONAL,0,2021-10-26 03:14:05,7.18,3.67,none,2021-10-26 03:14:08,37097.0,46083.0,0.0
1203715,20211025003926,85.95,23.68,2021102500,3926,NaN,46,2021-10-26 03:14:08,NaN,football,...,TRADITIONAL,0,2021-10-26 03:14:05,7.22,3.89,none,2021-10-26 03:14:08,37097.0,46083.0,0.0
1203716,20211025003926,85.95,23.68,2021102500,3926,NaN,47,2021-10-26 03:14:09,NaN,football,...,TRADITIONAL,0,2021-10-26 03:14:05,7.27,4.08,none,2021-10-26 03:14:08,37097.0,46083.0,0.0
1203717,20211025003926,85.95,23.68,2021102500,3926,NaN,48,2021-10-26 03:14:09,NaN,football,...,TRADITIONAL,0,2021-10-26 03:14:05,7.31,4.26,none,2021-10-26 03:14:08,37097.0,46083.0,0.0


In [120]:
#locations_with_flip.to_csv('data_flipped.csv', index = False)

In [3]:
classify = pd.read_csv('https://media.githubusercontent.com/media/cnickol26/BigDataBowl2023/main/data_flipped.csv')

<ipython-input-3-5a06576d793d>:1: DtypeWarning: Columns (22) have mixed types. Specify dtype option on import or set low_memory=False.
  classify = pd.read_csv('https://media.githubusercontent.com/media/cnickol26/BigDataBowl2023/main/data_flipped.csv')


In [13]:
## First going to remove any lineman who finsished the play past the line of scrimmage. These are most likely screen passes and we don't want to grade these plays. 
## Going to resplit it so it will be easier to remove everything associated with that specific lineman
flip = classify[((classify['Flip'] == 1) | (classify['pff_positionLinedUp'].isin(['QB_flip','ball_flip'])))]
flip

,uniqueplayId,football_x,football_y,gameId,playId,nflId,frameId,time,jerseyNumber,team,...,dropBackType,pff_playAction,ball_snap_time,new_x,new_y,chip_side,end_time,Right_End,Left_End,Flip
189199,2021090900187,78.21,26.39,2021090900,187,43424.0,1,2021-09-10 00:29:15,4.0,DAL,...,TRADITIONAL,0,2021-09-10 00:29:15,2.94,-0.06,none,2021-09-10 00:29:17,35441.0,35454.0,1.0
189200,2021090900187,78.21,26.39,2021090900,187,43424.0,2,2021-09-10 00:29:15,4.0,DAL,...,TRADITIONAL,0,2021-09-10 00:29:15,2.94,-0.07,none,2021-09-10 00:29:17,35441.0,35454.0,1.0
189201,2021090900187,78.21,26.39,2021090900,187,43424.0,3,2021-09-10 00:29:15,4.0,DAL,...,TRADITIONAL,0,2021-09-10 00:29:15,2.95,-0.07,none,2021-09-10 00:29:17,35441.0,35454.0,1.0
189202,2021090900187,78.21,26.39,2021090900,187,43424.0,4,2021-09-10 00:29:15,4.0,DAL,...,TRADITIONAL,0,2021-09-10 00:29:15,2.96,-0.07,none,2021-09-10 00:29:17,35441.0,35454.0,1.0
189203,2021090900187,78.21,26.39,2021090900,187,43424.0,5,2021-09-10 00:29:15,4.0,DAL,...,TRADITIONAL,0,2021-09-10 00:29:15,2.99,-0.08,none,2021-09-10 00:29:17,35441.0,35454.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1629274,20211025003926,85.95,23.68,2021102500,3926,NaN,45,2021-10-26 03:14:08,NaN,football,...,TRADITIONAL,0,2021-10-26 03:14:05,7.18,3.67,none,2021-10-26 03:14:08,37097.0,46083.0,0.0
1629275,20211025003926,85.95,23.68,2021102500,3926,NaN,46,2021-10-26 03:14:08,NaN,football,...,TRADITIONAL,0,2021-10-26 03:14:05,7.22,3.89,none,2021-10-26 03:14:08,37097.0,46083.0,0.0
1629276,20211025003926,85.95,23.68,2021102500,3926,NaN,47,2021-10-26 03:14:09,NaN,football,...,TRADITIONAL,0,2021-10-26 03:14:05,7.27,4.08,none,2021-10-26 03:14:08,37097.0,46083.0,0.0
1629277,20211025003926,85.95,23.68,2021102500,3926,NaN,48,2021-10-26 03:14:09,NaN,football,...,TRADITIONAL,0,2021-10-26 03:14:05,7.31,4.26,none,2021-10-26 03:14:08,37097.0,46083.0,0.0


In [21]:
flip2 = flip[(flip['time'] == flip['end_time']) & (flip['pff_positionLinedUp'].isin(['LT','RT']))]
flip2

,uniqueplayId,football_x,football_y,gameId,playId,nflId,frameId,time,jerseyNumber,team,...,dropBackType,pff_playAction,ball_snap_time,new_x,new_y,chip_side,end_time,Right_End,Left_End,Flip
1003091,202109090097,41.56,23.92,2021090900,97,52421.0,40,2021-09-10 00:26:35,78.0,TB,...,TRADITIONAL,0,2021-09-10 00:26:31,7.27,0.51,none,2021-09-10 00:26:35,53441.0,42403.0,1.0
1003092,202109090097,41.56,23.92,2021090900,97,52421.0,41,2021-09-10 00:26:35,78.0,TB,...,TRADITIONAL,0,2021-09-10 00:26:31,7.55,0.48,none,2021-09-10 00:26:35,53441.0,42403.0,1.0
1003093,202109090097,41.56,23.92,2021090900,97,52421.0,42,2021-09-10 00:26:35,78.0,TB,...,TRADITIONAL,0,2021-09-10 00:26:31,7.84,0.47,none,2021-09-10 00:26:35,53441.0,42403.0,1.0
1003094,202109090097,41.56,23.92,2021090900,97,52421.0,43,2021-09-10 00:26:35,78.0,TB,...,TRADITIONAL,0,2021-09-10 00:26:31,8.10,0.44,none,2021-09-10 00:26:35,53441.0,42403.0,1.0
1003168,2021090900137,108.68,23.85,2021090900,137,42654.0,32,2021-09-10 00:28:13,71.0,DAL,...,TRADITIONAL,0,2021-09-10 00:28:10,5.83,3.48,none,2021-09-10 00:28:13,40074.0,35454.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1629228,20211025003926,85.95,23.68,2021102500,3926,43447.0,42,2021-10-26 03:14:08,72.0,SEA,...,TRADITIONAL,0,2021-10-26 03:14:05,5.33,2.36,none,2021-10-26 03:14:08,37097.0,46083.0,1.0
1629229,20211025003926,85.95,23.68,2021102500,3926,43447.0,43,2021-10-26 03:14:08,72.0,SEA,...,TRADITIONAL,0,2021-10-26 03:14:05,5.44,2.47,none,2021-10-26 03:14:08,37097.0,46083.0,1.0
1629230,20211025003926,85.95,23.68,2021102500,3926,43447.0,44,2021-10-26 03:14:08,72.0,SEA,...,TRADITIONAL,0,2021-10-26 03:14:05,5.54,2.59,none,2021-10-26 03:14:08,37097.0,46083.0,1.0
1629231,20211025003926,85.95,23.68,2021102500,3926,43447.0,45,2021-10-26 03:14:08,72.0,SEA,...,TRADITIONAL,0,2021-10-26 03:14:05,5.62,2.72,none,2021-10-26 03:14:08,37097.0,46083.0,1.0


In [24]:
## Have multiple frames here since end time lasts a second but that is okay because we don't want any tackles who went across LOS a second or less before play 'end'
flip3 = flip2[flip2['new_y'] > 0]
flip3

,uniqueplayId,football_x,football_y,gameId,playId,nflId,frameId,time,jerseyNumber,team,...,dropBackType,pff_playAction,ball_snap_time,new_x,new_y,chip_side,end_time,Right_End,Left_End,Flip
1003091,202109090097,41.56,23.92,2021090900,97,52421.0,40,2021-09-10 00:26:35,78.0,TB,...,TRADITIONAL,0,2021-09-10 00:26:31,7.27,0.51,none,2021-09-10 00:26:35,53441.0,42403.0,1.0
1003092,202109090097,41.56,23.92,2021090900,97,52421.0,41,2021-09-10 00:26:35,78.0,TB,...,TRADITIONAL,0,2021-09-10 00:26:31,7.55,0.48,none,2021-09-10 00:26:35,53441.0,42403.0,1.0
1003093,202109090097,41.56,23.92,2021090900,97,52421.0,42,2021-09-10 00:26:35,78.0,TB,...,TRADITIONAL,0,2021-09-10 00:26:31,7.84,0.47,none,2021-09-10 00:26:35,53441.0,42403.0,1.0
1003094,202109090097,41.56,23.92,2021090900,97,52421.0,43,2021-09-10 00:26:35,78.0,TB,...,TRADITIONAL,0,2021-09-10 00:26:31,8.10,0.44,none,2021-09-10 00:26:35,53441.0,42403.0,1.0
1003168,2021090900137,108.68,23.85,2021090900,137,42654.0,32,2021-09-10 00:28:13,71.0,DAL,...,TRADITIONAL,0,2021-09-10 00:28:10,5.83,3.48,none,2021-09-10 00:28:13,40074.0,35454.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1629228,20211025003926,85.95,23.68,2021102500,3926,43447.0,42,2021-10-26 03:14:08,72.0,SEA,...,TRADITIONAL,0,2021-10-26 03:14:05,5.33,2.36,none,2021-10-26 03:14:08,37097.0,46083.0,1.0
1629229,20211025003926,85.95,23.68,2021102500,3926,43447.0,43,2021-10-26 03:14:08,72.0,SEA,...,TRADITIONAL,0,2021-10-26 03:14:05,5.44,2.47,none,2021-10-26 03:14:08,37097.0,46083.0,1.0
1629230,20211025003926,85.95,23.68,2021102500,3926,43447.0,44,2021-10-26 03:14:08,72.0,SEA,...,TRADITIONAL,0,2021-10-26 03:14:05,5.54,2.59,none,2021-10-26 03:14:08,37097.0,46083.0,1.0
1629231,20211025003926,85.95,23.68,2021102500,3926,43447.0,45,2021-10-26 03:14:08,72.0,SEA,...,TRADITIONAL,0,2021-10-26 03:14:05,5.62,2.72,none,2021-10-26 03:14:08,37097.0,46083.0,1.0


In [36]:
flip_ids = flip3['uniqueplayId']
flip_filtered = flip[flip['uniqueplayId'].isin(flip_ids)]
flip_filtered

,uniqueplayId,football_x,football_y,gameId,playId,nflId,frameId,time,jerseyNumber,team,...,dropBackType,pff_playAction,ball_snap_time,new_x,new_y,chip_side,end_time,Right_End,Left_End,Flip
189199,2021090900187,78.21,26.39,2021090900,187,43424.0,1,2021-09-10 00:29:15,4.0,DAL,...,TRADITIONAL,0,2021-09-10 00:29:15,2.94,-0.06,none,2021-09-10 00:29:17,35441.0,35454.0,1.0
189200,2021090900187,78.21,26.39,2021090900,187,43424.0,2,2021-09-10 00:29:15,4.0,DAL,...,TRADITIONAL,0,2021-09-10 00:29:15,2.94,-0.07,none,2021-09-10 00:29:17,35441.0,35454.0,1.0
189201,2021090900187,78.21,26.39,2021090900,187,43424.0,3,2021-09-10 00:29:15,4.0,DAL,...,TRADITIONAL,0,2021-09-10 00:29:15,2.95,-0.07,none,2021-09-10 00:29:17,35441.0,35454.0,1.0
189202,2021090900187,78.21,26.39,2021090900,187,43424.0,4,2021-09-10 00:29:15,4.0,DAL,...,TRADITIONAL,0,2021-09-10 00:29:15,2.96,-0.07,none,2021-09-10 00:29:17,35441.0,35454.0,1.0
189203,2021090900187,78.21,26.39,2021090900,187,43424.0,5,2021-09-10 00:29:15,4.0,DAL,...,TRADITIONAL,0,2021-09-10 00:29:15,2.99,-0.08,none,2021-09-10 00:29:17,35441.0,35454.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1629274,20211025003926,85.95,23.68,2021102500,3926,NaN,45,2021-10-26 03:14:08,NaN,football,...,TRADITIONAL,0,2021-10-26 03:14:05,7.18,3.67,none,2021-10-26 03:14:08,37097.0,46083.0,0.0
1629275,20211025003926,85.95,23.68,2021102500,3926,NaN,46,2021-10-26 03:14:08,NaN,football,...,TRADITIONAL,0,2021-10-26 03:14:05,7.22,3.89,none,2021-10-26 03:14:08,37097.0,46083.0,0.0
1629276,20211025003926,85.95,23.68,2021102500,3926,NaN,47,2021-10-26 03:14:09,NaN,football,...,TRADITIONAL,0,2021-10-26 03:14:05,7.27,4.08,none,2021-10-26 03:14:08,37097.0,46083.0,0.0
1629277,20211025003926,85.95,23.68,2021102500,3926,NaN,48,2021-10-26 03:14:09,NaN,football,...,TRADITIONAL,0,2021-10-26 03:14:05,7.31,4.26,none,2021-10-26 03:14:08,37097.0,46083.0,0.0


In [17]:
same = classify[((classify['Flip'] == 0) & ~(classify['pff_positionLinedUp'].isin(['QB_flip','ball_flip'])))]
same

,uniqueplayId,football_x,football_y,gameId,playId,nflId,frameId,time,jerseyNumber,team,...,dropBackType,pff_playAction,ball_snap_time,new_x,new_y,chip_side,end_time,Right_End,Left_End,Flip
0,202109090097,41.56,23.92,2021090900,97,42403.0,1,2021-09-10 00:26:31,94.0,DAL,...,TRADITIONAL,0,2021-09-10 00:26:31,2.47,8.77,none,2021-09-10 00:26:35,53441.0,42403.0,0.0
1,202109090097,41.56,23.92,2021090900,97,42403.0,2,2021-09-10 00:26:31,94.0,DAL,...,TRADITIONAL,0,2021-09-10 00:26:31,2.46,8.76,none,2021-09-10 00:26:35,53441.0,42403.0,0.0
2,202109090097,41.56,23.92,2021090900,97,42403.0,3,2021-09-10 00:26:31,94.0,DAL,...,TRADITIONAL,0,2021-09-10 00:26:31,2.43,8.75,none,2021-09-10 00:26:35,53441.0,42403.0,0.0
3,202109090097,41.56,23.92,2021090900,97,42403.0,4,2021-09-10 00:26:31,94.0,DAL,...,TRADITIONAL,0,2021-09-10 00:26:31,2.42,8.74,none,2021-09-10 00:26:35,53441.0,42403.0,0.0
4,202109090097,41.56,23.92,2021090900,97,42403.0,5,2021-09-10 00:26:31,94.0,DAL,...,TRADITIONAL,0,2021-09-10 00:26:31,2.40,8.73,none,2021-09-10 00:26:35,53441.0,42403.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
807553,20211025003926,85.95,23.68,2021102500,3926,NaN,45,2021-10-26 03:14:08,NaN,football,...,TRADITIONAL,0,2021-10-26 03:14:05,7.18,-3.67,none,2021-10-26 03:14:08,37097.0,46083.0,0.0
807554,20211025003926,85.95,23.68,2021102500,3926,NaN,46,2021-10-26 03:14:08,NaN,football,...,TRADITIONAL,0,2021-10-26 03:14:05,7.22,-3.89,none,2021-10-26 03:14:08,37097.0,46083.0,0.0
807555,20211025003926,85.95,23.68,2021102500,3926,NaN,47,2021-10-26 03:14:09,NaN,football,...,TRADITIONAL,0,2021-10-26 03:14:05,7.27,-4.08,none,2021-10-26 03:14:08,37097.0,46083.0,0.0
807556,20211025003926,85.95,23.68,2021102500,3926,NaN,48,2021-10-26 03:14:09,NaN,football,...,TRADITIONAL,0,2021-10-26 03:14:05,7.31,-4.26,none,2021-10-26 03:14:08,37097.0,46083.0,0.0


In [37]:
same2 = same[(same['time'] == same['end_time']) & (same['pff_positionLinedUp'].isin(['LT','RT']))]
same2

,uniqueplayId,football_x,football_y,gameId,playId,nflId,frameId,time,jerseyNumber,team,...,dropBackType,pff_playAction,ball_snap_time,new_x,new_y,chip_side,end_time,Right_End,Left_End,Flip
189013,202109090097,41.56,23.92,2021090900,97,42377.0,40,2021-09-10 00:26:35,76.0,TB,...,TRADITIONAL,0,2021-09-10 00:26:31,7.66,0.17,none,2021-09-10 00:26:35,53441.0,42403.0,0.0
189014,202109090097,41.56,23.92,2021090900,97,42377.0,41,2021-09-10 00:26:35,76.0,TB,...,TRADITIONAL,0,2021-09-10 00:26:31,7.73,0.11,none,2021-09-10 00:26:35,53441.0,42403.0,0.0
189015,202109090097,41.56,23.92,2021090900,97,42377.0,42,2021-09-10 00:26:35,76.0,TB,...,TRADITIONAL,0,2021-09-10 00:26:31,7.82,0.07,none,2021-09-10 00:26:35,53441.0,42403.0,0.0
189016,202109090097,41.56,23.92,2021090900,97,42377.0,43,2021-09-10 00:26:35,76.0,TB,...,TRADITIONAL,0,2021-09-10 00:26:31,7.92,0.03,none,2021-09-10 00:26:35,53441.0,42403.0,0.0
189090,2021090900137,108.68,23.85,2021090900,137,37082.0,32,2021-09-10 00:28:13,77.0,DAL,...,TRADITIONAL,0,2021-09-10 00:28:10,6.52,3.57,none,2021-09-10 00:28:13,40074.0,35454.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
807464,20211025003926,85.95,23.68,2021102500,3926,33107.0,42,2021-10-26 03:14:08,76.0,SEA,...,TRADITIONAL,0,2021-10-26 03:14:05,2.89,2.53,none,2021-10-26 03:14:08,37097.0,46083.0,0.0
807465,20211025003926,85.95,23.68,2021102500,3926,33107.0,43,2021-10-26 03:14:08,76.0,SEA,...,TRADITIONAL,0,2021-10-26 03:14:05,3.03,2.64,none,2021-10-26 03:14:08,37097.0,46083.0,0.0
807466,20211025003926,85.95,23.68,2021102500,3926,33107.0,44,2021-10-26 03:14:08,76.0,SEA,...,TRADITIONAL,0,2021-10-26 03:14:05,3.16,2.75,none,2021-10-26 03:14:08,37097.0,46083.0,0.0
807467,20211025003926,85.95,23.68,2021102500,3926,33107.0,45,2021-10-26 03:14:08,76.0,SEA,...,TRADITIONAL,0,2021-10-26 03:14:05,3.30,2.84,none,2021-10-26 03:14:08,37097.0,46083.0,0.0


In [38]:
same3 = same2[same2['new_y'] > 0]
same3

,uniqueplayId,football_x,football_y,gameId,playId,nflId,frameId,time,jerseyNumber,team,...,dropBackType,pff_playAction,ball_snap_time,new_x,new_y,chip_side,end_time,Right_End,Left_End,Flip
189013,202109090097,41.56,23.92,2021090900,97,42377.0,40,2021-09-10 00:26:35,76.0,TB,...,TRADITIONAL,0,2021-09-10 00:26:31,7.66,0.17,none,2021-09-10 00:26:35,53441.0,42403.0,0.0
189014,202109090097,41.56,23.92,2021090900,97,42377.0,41,2021-09-10 00:26:35,76.0,TB,...,TRADITIONAL,0,2021-09-10 00:26:31,7.73,0.11,none,2021-09-10 00:26:35,53441.0,42403.0,0.0
189015,202109090097,41.56,23.92,2021090900,97,42377.0,42,2021-09-10 00:26:35,76.0,TB,...,TRADITIONAL,0,2021-09-10 00:26:31,7.82,0.07,none,2021-09-10 00:26:35,53441.0,42403.0,0.0
189016,202109090097,41.56,23.92,2021090900,97,42377.0,43,2021-09-10 00:26:35,76.0,TB,...,TRADITIONAL,0,2021-09-10 00:26:31,7.92,0.03,none,2021-09-10 00:26:35,53441.0,42403.0,0.0
189090,2021090900137,108.68,23.85,2021090900,137,37082.0,32,2021-09-10 00:28:13,77.0,DAL,...,TRADITIONAL,0,2021-09-10 00:28:10,6.52,3.57,none,2021-09-10 00:28:13,40074.0,35454.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
807464,20211025003926,85.95,23.68,2021102500,3926,33107.0,42,2021-10-26 03:14:08,76.0,SEA,...,TRADITIONAL,0,2021-10-26 03:14:05,2.89,2.53,none,2021-10-26 03:14:08,37097.0,46083.0,0.0
807465,20211025003926,85.95,23.68,2021102500,3926,33107.0,43,2021-10-26 03:14:08,76.0,SEA,...,TRADITIONAL,0,2021-10-26 03:14:05,3.03,2.64,none,2021-10-26 03:14:08,37097.0,46083.0,0.0
807466,20211025003926,85.95,23.68,2021102500,3926,33107.0,44,2021-10-26 03:14:08,76.0,SEA,...,TRADITIONAL,0,2021-10-26 03:14:05,3.16,2.75,none,2021-10-26 03:14:08,37097.0,46083.0,0.0
807467,20211025003926,85.95,23.68,2021102500,3926,33107.0,45,2021-10-26 03:14:08,76.0,SEA,...,TRADITIONAL,0,2021-10-26 03:14:05,3.30,2.84,none,2021-10-26 03:14:08,37097.0,46083.0,0.0


In [39]:
same_ids = same3['uniqueplayId']
same_filtered = same[same['uniqueplayId'].isin(same_ids)]
same_filtered

,uniqueplayId,football_x,football_y,gameId,playId,nflId,frameId,time,jerseyNumber,team,...,dropBackType,pff_playAction,ball_snap_time,new_x,new_y,chip_side,end_time,Right_End,Left_End,Flip
0,202109090097,41.56,23.92,2021090900,97,42403.0,1,2021-09-10 00:26:31,94.0,DAL,...,TRADITIONAL,0,2021-09-10 00:26:31,2.47,8.77,none,2021-09-10 00:26:35,53441.0,42403.0,0.0
1,202109090097,41.56,23.92,2021090900,97,42403.0,2,2021-09-10 00:26:31,94.0,DAL,...,TRADITIONAL,0,2021-09-10 00:26:31,2.46,8.76,none,2021-09-10 00:26:35,53441.0,42403.0,0.0
2,202109090097,41.56,23.92,2021090900,97,42403.0,3,2021-09-10 00:26:31,94.0,DAL,...,TRADITIONAL,0,2021-09-10 00:26:31,2.43,8.75,none,2021-09-10 00:26:35,53441.0,42403.0,0.0
3,202109090097,41.56,23.92,2021090900,97,42403.0,4,2021-09-10 00:26:31,94.0,DAL,...,TRADITIONAL,0,2021-09-10 00:26:31,2.42,8.74,none,2021-09-10 00:26:35,53441.0,42403.0,0.0
4,202109090097,41.56,23.92,2021090900,97,42403.0,5,2021-09-10 00:26:31,94.0,DAL,...,TRADITIONAL,0,2021-09-10 00:26:31,2.40,8.73,none,2021-09-10 00:26:35,53441.0,42403.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
807553,20211025003926,85.95,23.68,2021102500,3926,NaN,45,2021-10-26 03:14:08,NaN,football,...,TRADITIONAL,0,2021-10-26 03:14:05,7.18,-3.67,none,2021-10-26 03:14:08,37097.0,46083.0,0.0
807554,20211025003926,85.95,23.68,2021102500,3926,NaN,46,2021-10-26 03:14:08,NaN,football,...,TRADITIONAL,0,2021-10-26 03:14:05,7.22,-3.89,none,2021-10-26 03:14:08,37097.0,46083.0,0.0
807555,20211025003926,85.95,23.68,2021102500,3926,NaN,47,2021-10-26 03:14:09,NaN,football,...,TRADITIONAL,0,2021-10-26 03:14:05,7.27,-4.08,none,2021-10-26 03:14:08,37097.0,46083.0,0.0
807556,20211025003926,85.95,23.68,2021102500,3926,NaN,48,2021-10-26 03:14:09,NaN,football,...,TRADITIONAL,0,2021-10-26 03:14:05,7.31,-4.26,none,2021-10-26 03:14:08,37097.0,46083.0,0.0


In [41]:
classify2 = pd.concat([same_filtered,flip_filtered])
classify2

,uniqueplayId,football_x,football_y,gameId,playId,nflId,frameId,time,jerseyNumber,team,...,dropBackType,pff_playAction,ball_snap_time,new_x,new_y,chip_side,end_time,Right_End,Left_End,Flip
0,202109090097,41.56,23.92,2021090900,97,42403.0,1,2021-09-10 00:26:31,94.0,DAL,...,TRADITIONAL,0,2021-09-10 00:26:31,2.47,8.77,none,2021-09-10 00:26:35,53441.0,42403.0,0.0
1,202109090097,41.56,23.92,2021090900,97,42403.0,2,2021-09-10 00:26:31,94.0,DAL,...,TRADITIONAL,0,2021-09-10 00:26:31,2.46,8.76,none,2021-09-10 00:26:35,53441.0,42403.0,0.0
2,202109090097,41.56,23.92,2021090900,97,42403.0,3,2021-09-10 00:26:31,94.0,DAL,...,TRADITIONAL,0,2021-09-10 00:26:31,2.43,8.75,none,2021-09-10 00:26:35,53441.0,42403.0,0.0
3,202109090097,41.56,23.92,2021090900,97,42403.0,4,2021-09-10 00:26:31,94.0,DAL,...,TRADITIONAL,0,2021-09-10 00:26:31,2.42,8.74,none,2021-09-10 00:26:35,53441.0,42403.0,0.0
4,202109090097,41.56,23.92,2021090900,97,42403.0,5,2021-09-10 00:26:31,94.0,DAL,...,TRADITIONAL,0,2021-09-10 00:26:31,2.40,8.73,none,2021-09-10 00:26:35,53441.0,42403.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1629274,20211025003926,85.95,23.68,2021102500,3926,NaN,45,2021-10-26 03:14:08,NaN,football,...,TRADITIONAL,0,2021-10-26 03:14:05,7.18,3.67,none,2021-10-26 03:14:08,37097.0,46083.0,0.0
1629275,20211025003926,85.95,23.68,2021102500,3926,NaN,46,2021-10-26 03:14:08,NaN,football,...,TRADITIONAL,0,2021-10-26 03:14:05,7.22,3.89,none,2021-10-26 03:14:08,37097.0,46083.0,0.0
1629276,20211025003926,85.95,23.68,2021102500,3926,NaN,47,2021-10-26 03:14:09,NaN,football,...,TRADITIONAL,0,2021-10-26 03:14:05,7.27,4.08,none,2021-10-26 03:14:08,37097.0,46083.0,0.0
1629277,20211025003926,85.95,23.68,2021102500,3926,NaN,48,2021-10-26 03:14:09,NaN,football,...,TRADITIONAL,0,2021-10-26 03:14:05,7.31,4.26,none,2021-10-26 03:14:08,37097.0,46083.0,0.0


In [64]:
## EDA looking at how to classify rush types
play1 = classify[classify['uniqueplayId'] == 2021091902389]
play1 = play1[play1['pff_positionLinedUp'].isin(['LT', 'RE'])]
play_data = play1[['team','new_x','new_y','o','frameId']]

In [65]:
def euclidean(a, b):
    return ((a[0] - b[0]) ** 2 + (a[1] - b[1]) ** 2) ** 0.5

In [68]:
play_data['dis'] = play_data.groupby(['frameId']).apply(lambda xy: xy and euclidean(xy, ref_xy))

ValueError: The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [71]:
test = play_data[play_data['frameId'] == 1]
test

,team,new_x,new_y,o,frameId
35057,CLE,0.75,3.55,78.29,1
304811,HOU,1.29,3.55,266.11,1
